In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt


import statsmodels.api as sm
import warnings
import math

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
df_cbs = pd.read_excel("attachments/DATASET_DISEASES_CBS.xlsx")
df_gemeentes = pd.read_excel("attachments/DATASET_GEMENEENTES_PC_4.xlsx")
df_gezondheid = pd.read_excel("attachments/DATASET_GEZONDHEIDSMONITOR.xlsx")
df_housing = pd.read_excel("attachments/DATASET_INHABITANTS_HOUSING.xlsx")
df_orv_gem = pd.read_excel("attachments/DATASET_ORV_GEMEENTE.xlsx")
df_orv_pc = pd.read_excel("attachments/DATASET_ORV_PC.xlsx")
df_demo = pd.read_excel("attachments/DATASET_SOCIO_DEMO.xlsx")



dfs = {"cbs": df_cbs,
       "gemeentes": df_gemeentes,
       "gezondheid": df_gezondheid,
       "housing": df_housing,
       "orv_gem": df_orv_gem,
       "orv_pc": df_orv_pc,
       "demo": df_demo}



#Function to parse special characters
def decode_unicode(value):
    if isinstance(value, str):
        # Replace all occurrences of _xXXXX_ with the corresponding character
        return re.sub(r'_x([0-9A-Fa-f]{4})_', lambda match: chr(int(match.group(1), 16)), value)
    return value


#General cleaning
dfs_precleaned = {}
for name, df in dfs.items():
    df.columns = df.columns.str.lower() #standardized the names
    df.rename(columns=lambda col: decode_unicode(col), inplace=True)
    df = df.dropna(axis = 0, how = 'any') #dropped NaN
    df = df.map(decode_unicode) #parsed special chars
    df = df.rename({"pc4": "postcode"}, axis = 1) #standardize the key name to "postcode"
    df = df.drop_duplicates()
    try:
        df['postcode'] = df['postcode'].astype(str).str.replace(".0",'') ##turn the postcode to string to use as join key
    except KeyError:
        pass

    dfs_precleaned[name] = df


#Table specific cleaning
dfs_precleaned['cbs'] = dfs_precleaned['cbs'].drop(columns = "diagnosis")

dfs_precleaned['gemeentes'] = dfs_precleaned['gemeentes'].drop(columns = "gemnaam")

dfs_precleaned['housing'] = dfs_precleaned['housing'].drop(["f34", "f35"], axis=1)

#ORV tables:
orv_postcode = pd.merge(dfs_precleaned['gemeentes'], dfs_precleaned['orv_pc'], on = "postcode")
orv_gemeente = dfs_precleaned['orv_gem'].copy()

In [ ]:
#Calculate the proportion of the population that are customers

#gemeente level:
orv_gemeente['customers_proportion'] = round(orv_gemeente.number_customers / orv_gemeente.inhabitants, 4)

#pc level:
orv_postcode['customers_proportion'] = round(orv_postcode.number_customers / orv_postcode.inhabitants, 4)

#We can set some kind of threshold for proportion of customers to call the region "under/over-represented" and then
#use it as a label for machine learning

orv_pc_mean = round(orv_postcode['customers_proportion'].mean(), 5)
orv_gem_mean = round(orv_gemeente['customers_proportion'].mean(), 5)
orv_pc_std = round(orv_postcode['customers_proportion'].std(), 5)
orv_gem_std = round(orv_gemeente['customers_proportion'].std(), 5)


#Standardize the proportions

orv_gemeente['customers_proportion_norm'] = orv_gemeente.customers_proportion.apply(lambda x: ((x - orv_gem_mean) / orv_gem_std))
orv_postcode['customers_proportion_norm'] = orv_postcode.customers_proportion.apply(lambda x: ((x - orv_pc_mean) / orv_pc_std))



In [ ]:
#CLASSIFY THE OBSERVATIONS
orv_gemeente['overrepresented'] = orv_gemeente['customers_proportion_norm'].apply(lambda x: 0 if x <= 0 else 1)
orv_postcode['overrepresented'] = orv_postcode['customers_proportion_norm'].apply(lambda x: 0 if x <= 0 else 1)

In [ ]:
#DO THE JOINS OF ORV DATA WITH OTHER TABLES TO GET ENRICHED DATASETS

#Join with demo df
demo_orv_pc = pd.merge(dfs_precleaned['demo'], orv_postcode, left_on = "postcode", right_on = "postcode")

#Join with housing df
housing = dfs_precleaned['housing'].copy()
housing_orv_pc = pd.merge(housing, orv_postcode, left_on= 'postcode', right_on = 'postcode')


#Join with gezondheid df

gezondhd = dfs_precleaned['gezondheid'].copy()

gezondheid_orv_pc = pd.merge(gezondhd, orv_postcode, left_on= 'postcode', right_on = 'postcode')


In [ ]:
#CREATE THE TABLE WITH ALL THE POSSIBLE FEATURES -> demo + housing + gezondheid + orv_pc
orv_postcode_to_join = orv_postcode.add_suffix('_orv')
orv_postcode_to_join = orv_postcode_to_join.rename(columns={"overrepresented_orv": "overrepresented"})
#add suffices to track the original dataset of a variable
demo_to_join = dfs_precleaned['demo'].add_suffix('_demo')
house_to_join = dfs_precleaned['housing'].add_suffix('_housing')
gezond_to_join = dfs_precleaned['gezondheid'].add_suffix('_gezond')


#Remove/transform columns
house_to_join['women_prop_housing'] = house_to_join['women_housing'] / house_to_join['total_housing']
house_to_join = house_to_join.drop(['total_housing', 'women_housing', 'men_housing'], axis = 1)

orv_total_join1 = pd.merge(demo_to_join, house_to_join, left_on = 'postcode_demo', right_on='postcode_housing')
orv_total_join2 = pd.merge(orv_total_join1, gezond_to_join, left_on = 'postcode_housing', right_on='postcode_gezond')
orv_total_join = pd.merge(orv_total_join2, orv_postcode_to_join, left_on = 'postcode_housing', right_on='postcode_orv')


to_drop_orv = [col for col in orv_total_join.columns if '_orv' in col]

to_drop_final = to_drop_orv + ["year_demo", "postcode_demo", "nr_households_demo", "postcode_housing", "postcode_gezond"]

orv_total_join_without_orv = orv_total_join.drop(to_drop_final, axis=1)


#ALTERNATIVE CLASSIFICATIONS:
#Multi-class with under, over and average (std)

to_drop_final.remove('customers_proportion_norm_orv')
orv_total_join_alter = orv_total_join.drop(to_drop_final, axis=1)

orv_total_join_alter['overrepresented_alt'] = orv_total_join_alter['customers_proportion_norm_orv'].apply( ####2 for over, 1 for under, 0 for normal
   lambda x: 2 if x > 1 else (1 if x < -1 else 0)
)

orv_total_join_alter = orv_total_join_alter.drop(["customers_proportion_norm_orv", "overrepresented"], axis = 1)
orv_total_join_alter = orv_total_join_alter.rename(columns={"overrepresented_alt": "overrepresented"})

In [ ]:
scaler = StandardScaler()

cols = list(orv_total_join.columns)

cols.remove("municipality_orv")


orv_total_join_standardized = pd.DataFrame(scaler.fit_transform(orv_total_join.drop(columns="municipality_orv")), columns=cols)

df = pd.concat([orv_total_join['municipality_orv'], orv_total_join_standardized], axis = 1)

In [ ]:
unstandardized_df = orv_total_join.drop(columns=df.filter(like='postcode').columns)


df_grouped = unstandardized_df.groupby('municipality_orv').mean()
df_grouped = df_grouped.sort_values(by = "customers_proportion_norm_orv")

df_bot = df_grouped.head(20)
df_top = df_grouped.tail(20)

dict_of_res = dict()
for col in df_grouped.columns:
    diff = abs(round(df_bot[col].mean(), 4) - round(df_top[col].mean(), 4))
    std = df_grouped[col].std()
    how_many = float(round(diff / std, 2))
    dict_of_res[col] = how_many



print(round(df_bot['age_from_65_housing'].mean(), 4))
print(round(df_top['age_from_65_housing'].mean(), 4))

In [ ]:
sorted_dict = dict(sorted(dict_of_res.items(), key=lambda item: item[1], reverse=True))

sorted_dict

*FEATURE SELECTION*

In [ ]:
#Define functions for ML dataset prep
def scale_dataset(dataframe, to_drop_additional = [], oversample=False):
  to_drop = ['year', 'postcode',	'nr_households', 'municipality',
                                'heavily_overrepresented_orv', 'heavily_underrepresented_orv',
                                "customers_proportion", "customers_proportion_norm", "total",
                                'inhabitants', 'number_customers'] + to_drop_additional
  for col in to_drop:
    try:
      dataframe = dataframe.drop(col, axis=1)
    except:
      pass


  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  X_df = dataframe[dataframe.columns[:-1]]

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  data = np.hstack((X, np.reshape(y, (-1, 1))))

  return data, X, y, X_df

In [ ]:
#Set up the CV
data_cv, X_cv, Y_cv, X_df = scale_dataset(orv_total_join_alter, oversample=False)

In [ ]:
#FEATURE IMPORTANCE

#RF MODEL
rf_model = RandomForestClassifier(
    n_estimators=400,       
    max_depth=4,            
    random_state=42,        
    #class_weight='balanced' 
)

rf_model.fit(X_cv, Y_cv)


#To differentiate between binary and multinomial log reg
if len(list(set(Y_cv))) == 2:
    lg_model = LogisticRegression()
else:
    lg_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

lg_model.fit(X_cv, Y_cv)


#BASE RF FEATURE IMPORTANCES
feature_importances = rf_model.feature_importances_

importance_rf_df = pd.DataFrame({
    'Feature': X_df.columns,
    'Importance_RF': feature_importances
}).sort_values(by='Importance_RF', ascending=False)



#PERMUTATION IMPORTANCE
from sklearn.inspection import permutation_importance

perm_result_rf = permutation_importance(rf_model, X_cv, Y_cv, n_repeats=10, random_state=42) #Based on random forest
perm_result_logreg = permutation_importance(lg_model, X_cv, Y_cv, n_repeats=10, random_state=42) #Based on log reg, either binomial or multinomial

perm_importance_df = pd.DataFrame({
    'Feature': X_df.columns,
    'Importance_Perm_RF': abs(perm_result_rf.importances_mean),
    'Importance_Perm_LR': abs(perm_result_logreg.importances_mean)
}).sort_values(by='Importance_Perm_RF', ascending=False)



#RFE
from sklearn.feature_selection import RFE

# With Logistic Regression as the estimator
lg_model = LogisticRegression(**lg_model.get_params()) #this resets the model so we can reuse it for RFE

estimator_logreg = lg_model
selector_logreg = RFE(estimator_logreg, n_features_to_select=30)
selector_logreg.fit(X_cv, Y_cv)

selected_vars_rfe_logreg = list(X_df.columns[selector_logreg.support_])

estimator_rf = RandomForestClassifier()
selector_rf = RFE(estimator_rf, n_features_to_select=30)
selector_rf.fit(X_cv, Y_cv)

selected_vars_rfe_rf = list(X_df.columns[selector_rf.support_])



In [ ]:
print(selected_vars_rfe_logreg)

print(selected_vars_rfe_rf)

#[x for x in selected_vars_rfe_rf if x in selected_vars_rfe_logreg]

#[x for x in selected_vars_rfe_rf if x not in selected_vars_rfe_logreg]

In [ ]:
importance = pd.merge(importance_rf_df, perm_importance_df, on = 'Feature')
#importance = importance.rename(columns={"Importance_x": "importance_random_forest", "Importance_y": "importance_permutation_method"})

#normalize both importance scores
scaler_new = StandardScaler()
importance['importance_rf_norm'] = scaler_new.fit_transform(importance[['Importance_RF']])
importance['importance_perm_rf_norm'] = scaler_new.fit_transform(importance[['Importance_Perm_RF']])
importance['importance_perm_lr_norm'] = scaler_new.fit_transform(importance[['Importance_Perm_LR']])

importance['importance_avg'] = (importance['importance_rf_norm'] + importance['importance_perm_rf_norm'] + importance['importance_perm_lr_norm']) / 3
importance = importance.sort_values(by = "importance_avg", ascending=False)

main_vars_importance = list(importance.head(30)['Feature'])

In [ ]:
#PCA

from sklearn.decomposition import PCA

# Initialize PCA
pca = PCA(n_components=X_df.shape[1])  # Use all components initially
pca.fit(X_cv)


loadings = pca.components_.T 
columns = X_df.columns

# Create a DataFrame to analyze loadings
loading_df = pd.DataFrame(loadings, columns=[f'PC{i+1}' for i in range(len(pca.components_))], index=columns)

important_vars_pc1 = loading_df['PC1'].abs().sort_values(ascending=False)

main_vars_pca = list(important_vars_pc1.index[:30])

In [ ]:
all_features = main_vars_pca + main_vars_importance + selected_vars_rfe_logreg + selected_vars_rfe_rf

#for var in unique_features:
counter = Counter(all_features)
selected_features = list({i for i in counter if counter[i] >= 3}) #A "consensus" filter, i.e. we leave the feature in only if it is present in at least 2 selection out of 4

In [ ]:
#SELECT THE FEATURES AND RUN NORMAL LOG REG


def run_log_reg(dataframe, multi=False):
    X_reg = dataframe[selected_features]
    y_reg = dataframe.iloc[:,-1]
    y_reg = y_reg.astype('category').cat.reorder_categories([0, 1, 2], ordered=True).cat.codes #explicitly set 0 ("Normal") as reference cat

    if multi:
        y_reg_encoded = y_reg.astype('category').cat.codes
        log_reg_multi = sm.MNLogit(y_reg_encoded, X_reg).fit()
        to_drop = ['t', "[0.025", "0.975]", "Std.Err."]

        results_reg1 = log_reg_multi.summary2().tables[1]
        results_reg1 = results_reg1.drop(columns = to_drop)

        results_reg2 = log_reg_multi.summary2().tables[2]
        results_reg2 = results_reg2.drop(columns = to_drop)

        results_reg2.columns = results_reg2.columns + "_2"
        results_reg = pd.concat([results_reg1, results_reg2], axis = 1)
    else:
        log_reg_binary = sm.Logit(y_reg, X_reg).fit()
        results_reg = log_reg_binary.summary2().tables[1]

    return results_reg



results_df_lr = run_log_reg(orv_total_join_alter, multi=True)
results_df_lr = results_df_lr.rename(columns={"y = 0": "Underrepresented vs Normal",
                                              "y = 1_2": "Overrepresented vs Normal",
                                              "P>|t|": "p-value",
                                              "P>|t|_2": "p-value2"})

results_df_lr['odds_ratio_y=1'] = pow(math.e, results_df_lr['Coef.'])
results_df_lr['odds_ratio_y=2'] = pow(math.e, results_df_lr['Coef._2'])

results_df_lr_filtered = results_df_lr[(results_df_lr['p-value'] < 0.05) | (results_df_lr['p-value2'] < 0.05)]

results_df_lr_filtered = results_df_lr_filtered.loc[results_df_lr_filtered['Underrepresented vs Normal'] != "ses_score_av_demo"]

results_df_lr_filtered['odds_ratio_y=1'] = np.exp(results_df_lr_filtered['Coef.'])
results_df_lr_filtered['odds_ratio_y=2'] = np.exp(results_df_lr_filtered['Coef._2'])

results_df_lr_filtered['Underrepresented vs Normal'] = results_df_lr_filtered['Underrepresented vs Normal'].str.replace(r"_[A-Za-z0-9]+$", '', regex=True)

In [ ]:
list(results_df_lr_filtered['Underrepresented vs Normal'])

In [ ]:
ax = results_df_lr_filtered.set_index('Underrepresented vs Normal')[['odds_ratio_y=1']].plot(kind='bar', figsize=(10, 6))

plt.axhline(y=1, color='gray', linestyle='--')
plt.ylabel('Odds Ratio')
plt.title('Odds Ratios for Underrepresented vs Normal representation (with SES score removed) ')
plt.legend().remove()

for rect in ax.patches:
    ax.text(rect.get_x() + rect.get_width() / 2, rect.get_height(),
            f'{rect.get_height():.2f}', ha='center', va='bottom')

plt.show()

In [ ]:
results_df_lr_filtered = results_df_lr[results_df_lr.iloc[:,-1] < 0.05]
results_df_lr_filtered.sort_values(by = list(results_df_lr_filtered.columns)[-1], ascending=False)

vars_selected_final = list(results_df_lr_filtered.index)
vars_selected_final.append('overrepresented')

ml_train_data = orv_total_join_alter[vars_selected_final]

*MACHINE LEARNING MODELS*

In [ ]:
data_ml, X_ml, Y_ml, X_df_ml = scale_dataset(ml_train_data, oversample=False)

In [ ]:
#Random forest
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

scoring_binary = ['accuracy', 'precision', 'recall', 'f1']

scoring_multinomial = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score, average='weighted'),
    'recall': make_scorer(recall_score, average='weighted'),
    'f1': make_scorer(f1_score, average='weighted')
}

3

rf_model_new = RandomForestClassifier(
    n_estimators=700,       # Number of trees in the forest
    max_depth=3,            # Maximum depth of each tree (None = fully grown trees)
    #random_state=42,        # For reproducibility
)
#rf_model_new.fit(X_ml, Y_ml)
cv_results_rf = cross_validate(rf_model_new, X_ml, Y_ml, cv=5, scoring=scoring_multinomial, return_train_score=True)

# for metric in scoring_binary:
#     print(f"{metric.capitalize()} (Mean): {np.mean(cv_results_rf[f'test_{metric}']):.2f}")

for metric in scoring_multinomial.keys():
    print(f"{metric.capitalize()} (Mean): {np.mean(cv_results_rf[f'test_{metric}']):.2f}")


In [ ]:
#LOG REG

if len(list(set(Y_ml))) == 2:
    lg_model = LogisticRegression()
else:
    lg_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')


# Perform cross-validation
cv_results_logistic = cross_validate(lg_model, X_ml, Y_ml, cv=5, scoring=scoring_multinomial, return_train_score=True)

for metric in scoring_multinomial:
    print(f"{metric.capitalize()} (Mean): {np.mean(cv_results_logistic[f'test_{metric}']):.2f}")

In [ ]:
#SVM
from sklearn.svm import SVC

svm_model = SVC(kernel='poly', random_state=42)

# Perform cross-validation
cv_results_svm = cross_validate(svm_model, X_ml, Y_ml, cv=5, scoring=scoring_multinomial, return_train_score=True)

# Print mean scores for each metric
for metric in scoring_multinomial:
    print(f"{metric.capitalize()} (Mean): {np.mean(cv_results_svm[f'test_{metric}']):.2f}")